# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install openai

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/pytorch/augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/pytorch/augmentation']


In [2]:
import re
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [3]:
df = fm.load(cfg.userdict_aug)

extension : .pickle
Loaded 121407 records from drive/MyDrive/MemeProject/data/augmentation/userdict_aug.pickle


# Reshape

In [4]:
df = df[['u', 'label']]

In [5]:
df.head()

,u,label
0,나 다시는 라면 안 먹을 거야!,98
1,와타시 다시는 라면 안 먹을 거야!,98
2,와따시 다시는 라면 안 먹을 거야!,98
3,나 중간고사 반에서 1등했어,147
4,와타시 중간고사 반에서 1등했어,147


In [6]:
df.shape

(121407, 2)

In [7]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df.shape

(108584, 2)

In [9]:
count = {i[1]['label']: 0 for i in df.iterrows()}

for i in df.iterrows():
  count[i[1]['label']] += 1

In [10]:
temp = sorted(count.items(), key=lambda x: x[1], reverse=True)

In [11]:
temp

[(18, 2319),
 (147, 1764),
 (57, 1699),
 (110, 1598),
 (51, 1581),
 (58, 1525),
 (108, 1519),
 (145, 1517),
 (106, 1441),
 (16, 1424),
 (28, 1334),
 (49, 1311),
 (66, 1170),
 (111, 1129),
 (73, 1114),
 (13, 1052),
 (44, 1025),
 (94, 1019),
 (128, 1008),
 (139, 997),
 (65, 976),
 (74, 970),
 (53, 970),
 (146, 964),
 (64, 962),
 (63, 954),
 (54, 945),
 (77, 934),
 (93, 926),
 (102, 923),
 (96, 916),
 (144, 908),
 (27, 902),
 (41, 891),
 (140, 888),
 (31, 879),
 (107, 873),
 (100, 869),
 (33, 862),
 (71, 858),
 (119, 855),
 (36, 855),
 (42, 848),
 (134, 842),
 (141, 828),
 (46, 825),
 (0, 819),
 (78, 812),
 (142, 796),
 (148, 790),
 (121, 789),
 (45, 772),
 (136, 771),
 (6, 770),
 (80, 768),
 (92, 765),
 (9, 763),
 (81, 763),
 (34, 756),
 (38, 749),
 (95, 736),
 (101, 734),
 (117, 728),
 (143, 725),
 (47, 720),
 (85, 719),
 (72, 719),
 (151, 718),
 (152, 712),
 (62, 705),
 (98, 704),
 (153, 704),
 (150, 700),
 (8, 691),
 (2, 691),
 (50, 678),
 (68, 672),
 (67, 671),
 (17, 663),
 (32, 659)

# Under sampling

In [12]:
df_over = df[df.label==18]
df_formal = df[df.label!=18]

In [13]:
df_formal.shape

(106265, 2)

In [14]:
class_over_temp = df_over.sample(frac=1).reset_index(drop=True)

In [15]:
class_over_temp = class_over_temp.iloc[:1700]

In [16]:
df_temp = pd.concat([class_over_temp, df_formal])

In [17]:
df_temp.reset_index(drop=True, inplace=True)

In [18]:
df_temp.head()

,u,label
0,나 클스마스에 애인이랑 남산타워 구경한다,18
1,오는 5/8에 크루즈여행 간다,18
2,5월 5일에 줄 조카 어린이날 선물 낼 산다,18
3,애인이랑 성탄절에 남산타워 오르기로 했어,18
4,크리스마스 때 최애 와인 개봉할 거야,18


# Save

In [19]:
df_temp.rename(columns={'u':'content'}, inplace=True)

In [20]:
fm.save(cfg.train_aug_data, df_temp)

Saved 107965 records
